# Measurements in systems with multiple qubits

In the previous tutorial, we discussed the concept of measurements done on single qubit systems. Building upon those ideas, this tutorial will introduce you to the concepts involved in, and the `Q#` implementation of measurements done on multi-qubit systems. This will involved measuring a single qubit in a multi-qubit system, as well as measuring multiple qubits simultaneously. 

We recommend to go through the [tutorial that introduces single qubit system measurements](../SingleQubitSystemMeasurements/SingleQubitSystemMeasurements.ipynb) before starting this one.

This tutorial covers the following topics for multiple qubit systems:

* Measuring qubits one at a time
* Measuring multiple qubits simultaneously
$\newcommand{\ket}[1]{\left|#1\right>}$
$\newcommand{\bra}[1]{\left<#1\right|}$

## Types of measurements
Measurements in an n-qubit system ($n>1$) are of the following types:
- Measuring all the qubits simultaneously in an orthogonal basis ($2^n$ possible outcomes) .
- Partial Measurements ($2^m$ possible outcomes, if $m<n$ number of qubits are measured).
- Joint Measurements ($2$ possible outcomes).

The number of outcomes for the three types of measurements are different. As we shall see below, the first type of measurement is a direct generalization of the concept of single qubit measurements introduced in the previous tutorial. Related to such measurements are partial measurements, which involve a partial collapse of the system's wavefunction since only some of the qubits are measured. On the other hand, joint measurements are usually done on all the qubits in a system simultaneously, but can result in only one out of two outcomes. 

We will discuss these concepts in the same order as in the list above.

## Full measurements: Measurements in multi-qubit bases
Consider a system consisting of $n\geq1$ qubits. The wavefunction of such a system belongs to a vector space of dimension $2^n$. Thus, the vector space is spanned by an orthogonal basis such as the computational basis which consists of the vectors $|0\dotsc0\rangle, \dotsc, |1\dotsc 1\rangle$. For generality, we consider an arbitrary orthonormal basis, which we denote by $\{ |b_0\rangle, |b_1\rangle, \dotsc, |b_{2^n-1}\rangle \}$. Then, the state $|\psi\rangle$ of the multi-qubit system can be expressed as a linear combination of the $2^n$ basis vectors. That is, there exist complex numbers $c_0,c_1,\dotsc, c_{2^n-1}$ such that
 $$
 |\psi\rangle = \sum_{i=0}^{2^n-1} c_i|b_i\rangle \equiv \begin{pmatrix}c_0 \\ c_1 \\ \vdots \\ c_{2^n-1}\end{pmatrix}.
 $$
 In line with the usual convention, we choose the wavefunction to be normalized, so that $|c_0|^2 + \dotsc + |c_{2^n-1}|^2 =1$. Then, a quantum measurement in the $\{ |b_0\rangle, |b_1\rangle, \dotsc, |b_{2^n-1}\rangle \}$ basis satisfies the following rules:
 - The measurement outcome $b_i$ occurs with probability $|c_i|^2$. 
 - Whenever the measurement outcome is $b_i$, the wavefunction collapses to the state $|b_i\rangle$. That is, the post measurement state of the system is equal to $|b_i\rangle$.

This can be summarized in the following table:
<table style="border:1px solid">
    <col width=150>
    <col width=150>
    <col width=150>
    <tr>
        <th style="text-align:center; border:1px solid">Measurement outcome</th>
        <th style="text-align:center; border:1px solid">Probability of outcome</th>
        <th style="text-align:center; border:1px solid">State after measurement</th>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid">$b_i$</td>
        <td style="text-align:center; border:1px solid">$|c_i|^2$</td>
        <td style="text-align:center; border:1px solid">$\ket{b_i}$</td>
    </tr>    
</table>
 
 
> Similar to measurements in single qubit systems, the assumption of normalization of the original wavefunction is required for the applicability of the statements above, in order to ensure that the sum of all the outcome probabilities is one.

### <span style="color:blue">Exercise 1</span>: Multi-qubit measurement outcome probabilities I 
Suppose that a two-qubit system is known to be in the following state:
$$\ket \psi =  \frac{1}{3}\ket {00} + \frac{2}{3} \ket {01} + \frac{2}{3}\ket {11}$$.

If all the qubits are measured simultaneously in the computational basis, what are the outcome probabilities?

*Can't come up with a solution? See the explained solution in the [--- Workbook](./Workbook_SingleQubitSystemMeasurements.ipynb#Exercise-1:-Obtain-the-probability-outcomes-for-a-specific-state).*


### <span style="color:blue">Exercise 2</span>: Multi-qubit measurement outcome probabilities II 
Suppose that a two-qubit system is known to be in the following state:
$$\ket \psi =  \frac{2}{3}\ket {00} + \frac{1}{3} \ket {01} + \frac{2}{3}\ket {11}$$.

If all the qubits are measured simultaneously in the Pauli X basis, i.e. in the $\{ \ket{++}, \ket{+-}, \ket{-+}, \ket{--}\}$ basis, what are the outcome probabilities?

*Can't come up with a solution? See the explained solution in the [--- Workbook](./Workbook_SingleQubitSystemMeasurements.ipynb#Exercise-1:-Obtain-the-probability-outcomes-for-a-specific-state).*


Although it is theoretically possible to implement measurement on all the qubits simultaneously as described in the previous sections, in Q\#, they are implemented by measuring all the qubits one after another.

## Measuring all qubits in a system one after another
As described in the previous sections, in theory, it is possible to measure all the qubits in an n-qubit system simultaneously in an orthogonal basis. The post-measurement state of the qubits is then exactly one out of the $2^n$ possible basis states.

In practice, this is implemented by measuring all the qubits one after another. For example, if one wants to measure a two-qubit system in the computational basis, one can implement this by first measuring the first qubit in the computational basis to obtain 0 or 1, followed by a computational basis measurement for the second qubit. This can result in one of the four possible outcomes: $\ket{00}, \ket{01}, \ket{10}, \ket{11}$.

This can be generalized to measurements in other bases, such as $\ket{++}, \ket{+-}, \ket{-+}, \ket{--}$, and for a larger number of qubits.

> The distinction between measuring all the qubits simultaneously, as opposed to one after another is not important for an ideal computer: in terms of the outcomes and measurement probabilities, both are identical. Furthermore, as long as all the qubits are measured, the sequence in which they are measured is also inconsequential. (However, these factors can be  important to consider for real quantum computers with imperfect qubits, but we restrict the discussion to ideal systems in this tutorial.) 

### <span style="color:blue">Demo: Measurement statistics for  qubit-by-qubit full measurement</span>
We demonstrate the equalivalence of the measurement probabilities for simultaneous measurements on all qubits, and measurements on all qubits executed one after another. Using the wavefuncion from exercise 1 above as an example, we show that the measurement probabilies obtained using the `M` operation in `Q#` are the same as those expected theoretically for exercise 1. 

In [2]:
// Run this cell using Ctrl+Enter (⌘+Enter on Mac)
// Run the next cell to see the output

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;

operation DemoBasisMeasurement(): Unit {
    
    // Define coefficients and obtain measurement probabilities
    // for the state corresponding to exercise 1
    // use little endian format
    let coefficients = [0.333, 0.0, 0.667, 0.667]; 
    let expected_probabilities = [0.111, 0.0, 0.445, 0.445];
    
    // Set up parameters and counter array for measurements
    mutable countArray = [0,0,0,0]; 
    let numRuns = 100;
    mutable littleEndianResult = 0; // measurement outcome for each iteration in little endian format
    mutable resultArray = new Result[2]; // measurement outcome for each iteration in Result[] format
    
    using (qs = Qubit[2]) {
        for (i in 1 .. numRuns) {
            // Prepare the state from Exercise 1:
            // |𝜓❭ = 0.33 |00❭ + 0.67 |01❭ + 0.67 |11❭ 
            // Use the PrepareArbitraryStateD operation for this purpose
            let Register = LittleEndian(qs);
            PrepareArbitraryStateD(coefficients, Register);
            if (i == 1) {
                Message("The state |𝜓❭ of the system before measurement is:");
                DumpMachine();
            } 
            
            // Measure the first qubit, followed by the second qubit
            for (j in 0 .. 1){
                set resultArray w/= j <- M(qs[j]);
            }
            set littleEndianResult = ResultArrayAsInt(resultArray);
            
            // update countArray
            set countArray w/= littleEndianResult <- countArray[littleEndianResult] + 1; 
            ResetAll(qs);
        }
    }
    
    // obtain simulated probability of measurement for each outcome
    mutable simulated_probabilities = new Double[4];
    for (i in 0 .. 3){
        set simulated_probabilities w/= i <- DividedByD(IntAsDouble(countArray[i]), IntAsDouble(numRuns));
    }
    
    Message($"Theoretical measurement probabilities are {expected_probabilities}");
    Message($"Simulated measurement probabilities are {simulated_probabilities}");
}

DemoBasisMeasurement

In [3]:
%simulate DemoBasisMeasurement

The state |𝜓❭ of the system before measurement is:


|0⟩	0.3328890000246523 + 0𝑖
|1⟩	1.962615573354719E-17 + 0𝑖
|2⟩	0.6667776667160455 + 0𝑖
|3⟩	0.6667776667160455 + 0𝑖

Theoretical measurement probabilities are [0.111,0,0.445,0.445]
Simulated measurement probabilities are [0.1,0,0.42,0.48]


()

The simulated probabilities will be different for each run of `DemoBasisMeasurement`. The simulated and theoretical probabilities are not expected to be identical, since measurements are probabilistic. However, we expect the values to be similar. Additionally on average, we expect the simulated probabilities to approach the theoretical probabilities, as `numRuns` is increased.

Full measurements can also be used in certain cases to identify the state of the system, if it is guaranteed to be in one out of some possible states.

### <span style="color:blue">Exercise 3</span>: Identify computational basis states

**Input**: An array of qubits which is guaranteed to be in a computational basis state.

**Output**: An array of `Result` values, with the same length as the number of qubits, specifying the computational basis state corresponding to the input state. For example, if the input qubit wavefunction is set to $\ket{001}$, the output should be the array `[Zero, Zero, One]`.

In [ ]:
// %kata T1_ReturnBasisState

operation ReturnBasisState(qs : Qubit[]) : Result[] {
    
    let N = Length(qs);
    mutable array = new Result[N];
    
    // ... 
    
    // for (i in 0 .. N - 1) {
    //    set array w/= i <- M(qs[i]);
    // }
    // return array;
}

## Partial Measurements

For a system with $n>1$ number of qubits, it is possible to measure $m<n$ number of qubits one after another. The number of measurement outcomes is then $2^m$ instead of $2^n$. The probabilities of each of the outcomes, and the post-measurement states of the qubits can be found using the projection formalism for measurements.

First, we recall the concept of [projecion operators](../SingleQubitSystemMeasurements/SingleQubitSystemMeasurements.ipynb#Measurements-as-projection-operations) introduced in the single qubit systems measurements tutorial. Measurements are modelled by orthogonal projection operators, which are matrices which satisfy
$$
P^2 = P^\dagger = P.
$$
Consider an $n$ qubit system which is in a state $|\psi\rangle$, for which the $m<n$ qubits are measured in an orthogonal basis $\{ |b_0\rangle , |b_1\rangle, \dotsc, |b_{2^m-1}\rangle\}$ corresponding to the $m$ qubits being measured. Then we define $2^m$ projectors, corresponding to each of the $|b_i\rangle$ states, as
$$
P_i = |b_i\rangle \langle b_i| \otimes \mathbb{1}_{n-m},
$$
where $\mathbb{1}_{n-m}$ is the identity operator over the remaining $(n-m)$ qubits. 
> The symbol $\otimes$ represents the tensor product or the Kroneker product of two matrices. It is different from the usual matrix multiplication. In the current context, $|b_i\rangle \langle b_i| \otimes \mathbb{1}_{n-m}$ simply means that the operator $|b_i\rangle \langle b_i|$ acts only of the $m$ qubits being measured, while the effect of $P_i$ on the remaining qubits is $\mathbb{1}_{n-m}$, i.e. the identity operator. We recommend revisiting the [Linear Algebra tutorial](../LinearAlgebra/LinearAlgebra.ipynb#Tensor-Product) for a refresher.

Analogous to the case for measurements for single qubit systems, the rules for partial measurement probabilies and outcomes can be summarized as follows:
- A measurement is done by choosing one of these projectors randomly.
- For eveny $i\in \{0,\dotsc,2^m-1$, $P_i$ is chosen with probability $|P_i|\psi\rangle|^2$.
- If $P_i$ is chosen, the measurement outcome is $b_i$, and the state of the system after the measurement is given by
$$
\frac{P_i |\psi\rangle}{|P_i |\psi\rangle|}.
$$

For example, consider a two-qubit system in the state $\ket \psi = \frac{1}{\sqrt{2}}\ket{01} - \frac{1}{\sqrt 2}\ket{10}$. Consider a measurement of the first qubit in the computational basis, i.e. in the $\{\ket 0 , \ket 1 \}$ basis. Then, we have two projectors, given by
\begin{align*}
P_0 &= \ket 0\bra 0 \otimes \mathbb{1},\\
P_1 &= \ket 1 \bra 1 \otimes \mathbb{1}.
\end{align*}
The action of $P_0$ on $\ket \psi$ is 
\begin{align*}
P_0 \ket \psi &= \left(\ket 0\bra 0 \otimes \mathbb{1}\right) \frac{1}{\sqrt 2}\big(\ket{01} - \ket{10}\big) \\
              &= \frac{1}{\sqrt 2} \big( \ket 0\bra 0 0\rangle \otimes \mathbb{1} \ket{1} - \ket 0 \bra 0 1\rangle \otimes \mathbb{1} \ket 0 \big) \\
              &= \frac{1}{\sqrt 2} \ket{01}.
\end{align*}
Similarly, we obtain 
$$
P_1 \ket\psi = -\frac{1}{\sqrt 2} \ket{10}.
$$
Clearly, we have $|P_0 \ket \psi| = |P_1 \ket \psi| = \frac{1}{2}$ in this case. Thus, the probabilities of measuring $0$ and $1$ are both $0.5$, with the post-measurement states of system being $\ket{01}$ and $\ket{10}$ respectively.

> Similar to the case of single qubit system measurements, the applicability of the formalism above requires the state of the multi-qubit system, $\ket \psi$, to be normalized. This is required to ensure that all the probability outcomes add to unity.

### <span style="color:blue">Exercise 4</span>: Partial measurement probabilities for the Hardy state

Consider a 2-qubit system which is in the state $\ket \psi = \frac{1}{\sqrt{12}} \big(3|00\rangle + |01\rangle + |10\rangle + |11\rangle\big)$.

If only the first qubit is measured in the computational basis what are the probabilities of the outcomes, and the post-measurement states of the system?

*Can't come up with a solution? See the explained solution in the [--- Workbook](./Workbook_SingleQubitSystemMeasurements.ipynb#Exercise-1:-Obtain-the-probability-outcomes-for-a-specific-state).*


### <span style="color:blue">Demo: Measurement statistics for  partial measurement</span>
Using the `M` operation in `Q#`, we demonstrate that the simulated outcome probabilities and post-measurement outcomes match with the theoretically expected values obtained using the projection operators as described above. We use the state from Exercise 4 with a computational basis measurement on the first qubit for this purpose.

In [1]:
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;

operation DemoPartialMeasurement(): Unit {
    let divider = "--------------------------------------------------------------------------------------------------";
    // 
    let expected_probabilities = [0.833, 0.167];
    
    // Set up parameters and counter array for measurements
    mutable countArray = [0,0];
    let numRuns = 100;
    
    using (qs = Qubit[2]) {
        for (i in 1 .. numRuns) {
            // Prepare the state from Exercise 4:
            // |𝜓❭ = (1/√12)(3|00⟩+|01⟩+|10⟩+|11⟩) 
            Ry(2.0 * ArcCos(Sqrt(10.0 / 12.0)), qs[0]);
            (ControlledOnInt(0, Ry))([qs[0]], (2.0 * ArcCos(Sqrt(9.0 / 10.0)), qs[1]));
            (ControlledOnInt(1, Ry))([qs[0]], (2.0 * PI()/4.0, qs[1]));
            
            if (i == 1) {
                Message("The state |𝜓❭ of the system before measurement is:");
                DumpMachine();
                Message(divider);
            } 
            
            // Measure the first qubit
            if (M(qs[0]) == Zero){
                set countArray w/= 0 <- countArray[0] + 1;
                if (countArray[0] == 1){ // for the first time the outcome is Zero
                    Message("For outcome 0, the post-measurement system of the system is:");
                    DumpMachine();
                }
            } else{
                set countArray w/= 1 <- countArray[1] + 1;
                if (countArray[1] == 1){ // for the first time the outcome is One
                    Message("For outcome 1, the post-measurement system of the system is:");
                    DumpMachine();
                }
            }
            ResetAll(qs);
        }
    }
    
    // obtain simulated probability of measurement for each outcome
    mutable simulated_probabilities = new Double[2];
    for (i in 0 .. 1){
        set simulated_probabilities w/= i <- DividedByD(IntAsDouble(countArray[i]), IntAsDouble(numRuns));
    }
    
    Message($"Theoretical measurement probabilities are {expected_probabilities}");
    Message($"Simulated measurement probabilities are {simulated_probabilities}");
}    


DemoPartialMeasurement

In [2]:
%simulate DemoPartialMeasurement

The state |𝜓❭ of the system before measurement is:


|0⟩	0.8660254037844387 + 0𝑖
|1⟩	0.2886751345948128 + 0𝑖
|2⟩	0.28867513459481303 + 0𝑖
|3⟩	0.2886751345948128 + 0𝑖

--------------------------------------------------------------------------------------------------
For outcome 1, the post-measurement system of the system is:


|0⟩	0 + 0𝑖
|1⟩	0.7071067811865476 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	0.7071067811865476 + 0𝑖

For outcome 0, the post-measurement system of the system is:


|0⟩	0.9486832980505138 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0.31622776601683805 + 0𝑖
|3⟩	0 + 0𝑖

Theoretical measurement probabilities are [0.833,0.167]
Simulated measurement probabilities are [0.86,0.14]


()

> The simulated and theoretical measurement probabilities are not expected to match exactly, but should be close to each other. However, the post-measurement outcomes from the simulation should match with the answer for Exercise 4.

In certain situations, it is possible to distinguish between orthogonal states of multi-qubit systems using partial measurements, as illustrated in the next exercise.
### <span style="color:blue">Exercise 5</span>: Distinguish orthogonal states

**Input**: A 3-qubit system which is guaranteed to be in either the $\ket{++-}$ state, or the $\ket{---}$ state.

**Output**: `true` if the qubit is in the $\ket{++-}$ state, or `false` if it was in the $\ket{---}$ state. The state of the qubits at the end of the operation should be the same as the initial state.

In [14]:
// %kata T2_DistinguishOrthogonalStates

operation DistinguishOrthogonalStates (qs : Qubit[]) : Bool {
    // ...
    return Measure([PauliX], [qs[0]]) == Zero;
}

DistinguishOrthogonalStates

## Measurements and entanglement

Whether qubits are entangled or not has an effect on the measurement statistics of the system. In particular, if two qubits are entangled, then their measurement outcomes will be correlated, while separable states (which are by definition not entangled) have uncorrelated measurement outcomes.

It is useful to revisit the concepts of entanglement and separable states, which were introduced in the [tutorial on multi-qubit systems](../MultiQubitSystems/MultiQubitSystems.ipynb#Entanglement). Consider a system of $n>1$ number of qubits, which we divide into two parts: A, consisting of $m$ qubits, and B consisting of the remaining $n-m$ qubits. We say that the state $\ket \psi$ of the composite (i.e. the entire) system is separable if it can be expressed as 
$$
\ket \psi = \ket {\phi_A} \otimes \ket{\phi_B}.
$$
where $\ket{\phi_A}$ and $\ket{\phi_B}$ are wavefunctions corresponding to parts $A$ and $B$ respectively. If it is not possible to express $\ket \psi$ in such a form, then we say that system A is entangled with system B.

Consider a measurement on the subsystem $A$ of a separable state. Let the measurement be done in a basis $\{ \ket{b_0},\dotsc,\ket{b_{2^m-1}}\}$. According to the projection formalism, a projection operator $P_i = \ket{b_i}\bra{b_i} \otimes \mathbb{1}$ randomly. The corresponding post-measurement state of the system is then given by
\begin{align*}
\ket{\psi}_{i} &\equiv \frac{P_i \ket{\psi}}{|P_i \ket{\psi}|} \\
               &= \frac{\ket{b_i}\bra{b_i}\phi_A\rangle \otimes \ket {\phi_B}}{|\ket{b_i}\bra{b_i}\phi_A\rangle \otimes \ket {\phi_B}|} \\
               &= \ket{b_i} \otimes \ket{\phi_B}.
\end{align*}
Thus, the state of subsystem $B$ is $\ket{\phi_B}$ independent of the outcome $i$ of the measurement on the first qubit. Thus, a subsequent measurement on subsystem $B$ (including outcome probabilities) is independent of the result of the first measurement. In other words, the outcomes of the two measurements will be uncorrelated.

On the other hand, if the system is entangled, then the measurement outcomes will be correlated, in a manner dictated by the bases chosen for the measurements on the two subsystems. The following exercise illustrates this phenomenon.





### <span style="color:blue">Exercise 6</span>: Sequential measurements on an entangled state and a separable state
Consider two two-qubit states:
- The Bell state $|\Phi^{+}\rangle = \frac{1}{\sqrt{2}} \big (|00\rangle + |11\rangle\big)$.
- A state $\ket \Theta = \frac{1}{2} \big( \ket{00} + \ket{01} + \ket{10} + \ket{11} \big)$.

For both the states, consider a measurement on the first qubit, followed by a measurement on the second second qubit (both in the computational basis). For which state can we expect the measurement outcomes to be correlated? Verify by calculating the sequential measurement probabilities explicitly for both the states. 

<br/>
<details>
    <summary><strong>Answer: to be moved to the workbook</strong></summary>
- *The Bell state*: If the first qubit is measured to be in the state $\ket 0$, a subsequent measurement on the second qubit *always* results in an outcome of $0$, with probabilitiy $1$. Similarly, if the outcome of the measurement on the first qubit is $1$, then the second qubit measurement always results in $1$. Thus, sequential measurements are perfectly *correlated*.
- *Separable state* $\ket \Theta$: Irrespective of whether the first qubit measurement outcome is $0$ of $1$ (each of which occurs with a probability of $0.5$), as subsequent measurement on the second qubit results in an outcome of $0$ or $1$ (both with a probabilitiy of $0.5$). Thus, sequential measurements are perfectly *uncorrelated*.  
</details>

## Joint measurements

Joint measurements, also known as Pauli measurements are a generalization of full and partial measurements to other bases. In `Q#`, joint measurements are implemented using the [Measure](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.measure) operation. It is instructive to review single qubit measurements in a different light, before discussing joint measurements. 

### Single-qubit Pauli measurement
For a single qubit systems, any measurement corresponding to an orthogonal basis can be considered to be associated with a Hermitian matrix with eigenvalues $\pm 1$. The possible measurement outcomes (analogous to `Result` in `Q#`) are the eigenvalues of the Hermitian matrix, and the corresponding projection matrices for the measurement are the projection operators onto the *eigenspaces* corresponding to the eigenvalues. 

For example, consider the computational basis measurement, which can result in outcomes `Zero` or `One` corresponding to states $\ket 0$ and $\ket 1$. This measurement is associated with the Pauli Z operator, which is given by 
$$
Z = \begin{pmatrix} 1 & 0 \\ 0 & -1\end{pmatrix} \equiv \ket{0}\bra{0} - \ket{1}\bra{1}.
$$
The $Z$ operator has two eigenvalues, $1$ and $-1$, with corresponding eigenvectors $\ket{0}$ and $\ket{1}$. A $Z$-measurement is then a measurement in the $\{\ket{0},\ket{1}\}$ basis, with the measurement outcomes being $1$ nad $-1$ respectively. In `Q#`, by convention, an eigenvalue of $1$ corresponds to a `Result` of `Zero`, while an eigenvalue of $-1$ corresponds to a `Result` of `One`.

Similarly, one can implement measurements corresponding to the Pauli X and Y operators. We summarize the various properties below:
<table style="border:1px solid">
    <col width=200>
    <col width=50>
    <col width=100>
    <col width=150>
    <col width=150>
    <tr>
        <th style="text-align:center; border:1px solid">Pauli Operator</th>
        <th style="text-align:center; border:1px solid">Matrix</th>
        <th style="text-align:center; border:1px solid">Eigenvalue</th>
        <th style="text-align:center; border:1px solid">Eigenvector/post-measurement state</th>
        <th style="text-align:center; border:1px solid">Measurement Result in Q#</th>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid" rowspan="2">$X$</td>
        <td style="text-align:center; border:1px solid" rowspan="2">$\begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$</td>
        <td style="text-align:center; border:1px solid">+1</td>
        <td style="text-align:center; border:1px solid">$\ket{+}$</td>
        <td style="text-align:center; border:1px solid">Zero</td>
    </tr><tr>
        <td style="text-align:center; border:1px solid">-1</td>
        <td style="text-align:center; border:1px solid">$\ket{-}$</td>
        <td style="text-align:center; border:1px solid">One</td>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid" rowspan="2">$Y$</td>
        <td style="text-align:center; border:1px solid" rowspan="2">$\begin{bmatrix} 0 & -i \\ i & 0 \end{bmatrix}$</td>
        <td style="text-align:center; border:1px solid">+1</td>
        <td style="text-align:center; border:1px solid">$\ket{i}$</td>
        <td style="text-align:center; border:1px solid">Zero</td>
    </tr><tr>
        <td style="text-align:center; border:1px solid">-1</td>
        <td style="text-align:center; border:1px solid">$\ket{-i}$</td>
        <td style="text-align:center; border:1px solid">One</td>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid" rowspan="2">$Z$</td>
        <td style="text-align:center; border:1px solid" rowspan="2">$\begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}$</td>
        <td style="text-align:center; border:1px solid">+1</td>
        <td style="text-align:center; border:1px solid">$\ket{0}$</td>
        <td style="text-align:center; border:1px solid">Zero</td>
    </tr><tr>
        <td style="text-align:center; border:1px solid">-1</td>
        <td style="text-align:center; border:1px solid">$\ket{1}$</td>
        <td style="text-align:center; border:1px solid">One</td>
    </tr>
</table>


In general, any measurement which results in two outcomes is a measurement corresponding to the Hermitian operator $U Z U^\dagger$, for some $U$ which is a $2\times 2$ unitary matrix.

## Multi-qubit Pauli measurements
Pauli measurements can also be generalized to a larger number of qubits. A Pauli measurement can correspond to an operator $M_1 \otimes \dotsc, M_n$ with each $M$ being from the set of gates $\{X,Y,Z,I\}$. If at least one of the operators is not the identity matrix, then the measurement can result in two outcomes: a `Result` of `Zero` corresponding to eigenvalue $1$ and a `Result` of `One` corresponding to the eigenvalue $-1$. The corresponding projection operators are the projections onto the corresponding eigenspaces.

For example, a Pauli/joint measurement corresponding to the $X\otimes Z$ operator can be characterized as follows:
<table style="border:1px solid">
    <col width=50>
    <col width=50>
    <col width=150>
    <col width=250>
    <tr>
        <th style="text-align:center; border:1px solid">Eigenvalue</th>
        <th style="text-align:center; border:1px solid">Measurement Result in Q#</th>
        <th style="text-align:center; border:1px solid">Eigenbasis</th>
        <th style="text-align:center; border:1px solid">Measurement Projector</th>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid">$+1$</td>
        <td style="text-align:center; border:1px solid">Zero</td>
        <td style="text-align:center; border:1px solid">$\{ \ket{0,+}, \ket{1,-} \}$</td>
        <td style="text-align:center; border:1px solid">$P_{+1} = \ket{0,+}\bra{0,+} +  \ket{1,-} \bra{1,-}$</td>
     </tr>
    <tr>
        <td style="text-align:center; border:1px solid">$-1$</td>
        <td style="text-align:center; border:1px solid">One</td>
        <td style="text-align:center; border:1px solid">$\{ \ket{0,-}, \ket{1,+} \}$</td>
        <td style="text-align:center; border:1px solid">$P_{-1} = \ket{0,-}\bra{0,-} +  \ket{1,+} \bra{1,+}$</td>
     </tr>
 </table>   
 
 The rules for measurements are then the same as those outlined in the [partial measurements section](#Partial-Measurements), with the projection operators in the table.

### <span style="color:blue">Exercise 7</span>: Joint measurement on a two-qubit system

Consider a Pauli measurement corresponding to $Y\otimes Z$, on the Bell state $|\Phi^{+}\rangle = \frac{1}{\sqrt{2}} \big (|00\rangle + |11\rangle\big)$.

What are the probabilities of the two outcomes, and the corresponding post-measurement states of the system?

### <span style="color:blue">Exercise 8</span>: Parity Measurement

**Inputs**: Two qubits (stored in an array) which are guaranteed to be either in superposition of states $|00\rangle$ and $|11\rangle$ or in superposition of states $|01\rangle$ and $|10\rangle$.

**Output**: 0 if qubits were in the first superposition, 1 if they were in the second superposition.  
*The state of the qubits at the end of the operation should be the same as the starting state.*

In [ ]:
%kata T03_ParityMeasurement 

operation ParityMeasurement (qs : Qubit[]) : Int {
    // ...
    return -1;
}